<a href="https://colab.research.google.com/github/simulate111/Data-Science_2024/blob/main/Mini%20Project%201%20%E2%80%93%20What%20should%20I%20cook%20tonight.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [97]:
import requests
from bs4 import BeautifulSoup
import logging

logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')

def scrape_recipe_details(link):
    headers = {'User-Agent': 'Mozilla/5.0'}
    try:
        response = requests.get(link, headers=headers, timeout=10)
        response.raise_for_status()  # Raise an error for bad responses
    except requests.RequestException as e:
        logging.error(f"Failed to retrieve recipe: {link} - {e}")
        return {}

    soup = BeautifulSoup(response.text, 'html.parser')

    try:
        title = soup.select_one('.entry-title')
        image = soup.select_one('.wp-block-image img')
        ww_points = soup.select_one('.recipe-meta-value.recipe-ww-pp')

        # Extracting titles from the links in the new class structure
        recipe_titles = soup.select('ul.cat-icons.post-icons a')
        recipe_keys_list = [key['title'] for key in recipe_titles if 'title' in key.attrs]

        calories_container = soup.select_one('.wprm-nutrition-label-text-nutrition-container-calories')
        calories_value = (calories_container.select_one('.wprm-nutrition-label-text-nutrition-value').get_text(strip=True)
                          if calories_container else "N/A")

        # Only return data if calories are not "N/A"
        if calories_value == "N/A":
            logging.info(f"Skipping recipe due to N/A calories: {link}")
            return {}

        title = title.get_text(strip=True) if title else "N/A"
        image_src = image['src'] if image else "N/A"
        points_value = ww_points.get_text(strip=True) if ww_points else "N/A"  # Renamed variable

        logging.debug(f"Title: {title}")
        logging.debug(f"Image: {image_src}")
        logging.debug(f"Points: {points_value}")  # Updated log message
        logging.debug(f"Recipe Keys: {recipe_keys_list}")  # Updated log message
        logging.debug(f"Calories: {calories_value}")

        return {
            'Name of the Food': title,
            'Image of the Food': image_src,
            'Calories': calories_value,
            'Points': points_value,
            'The recipe Key': recipe_keys_list  # Changed back to "The recipe Key"
        }
    except Exception as e:
        logging.error(f"Error extracting data from {link}: {e}")
        return {}


In [ ]:
# Scrape the first 5 pages
all_recipes = []
for page in range(1, 2):
    recipes = scrape_skinnytaste(page)
    all_recipes.extend(recipes)

Fetching page 1, status code: 200
Processing link: https://www.skinnytaste.com/free-7-day-healthy-meal-plan-sept-23-29/
Processing link: https://www.skinnytaste.com/apple-butter-recipe/
Processing link: https://www.skinnytaste.com/roasted-delicata-squash/
Processing link: https://www.skinnytaste.com/applesauce-nut-bread/
Processing link: https://www.skinnytaste.com/autumn-salad-with-pears-and-gorgonzola/
Processing link: https://www.skinnytaste.com/sesame-chicken/
Processing link: https://www.skinnytaste.com/bacon-in-the-oven/
Processing link: https://www.skinnytaste.com/skinny-pumpkin-spiced-latte/
Processing link: https://www.skinnytaste.com/homemade-hamburger-helper/
Processing link: https://www.skinnytaste.com/fried-brown-rice/
Processing link: https://www.skinnytaste.com/easy-sheet-pan-dinners/
Processing link: https://www.skinnytaste.com/asian-grilled-chicken-33-pts_3510/
Processing link: https://www.skinnytaste.com/sweet-potato-salad/
Processing link: https://www.skinnytaste.com

In [ ]:


# Check the contents of all_recipes
print(f"Total recipes collected: {len(all_recipes)}")  # Debugging line

# Create a DataFrame and display the results
df = pd.DataFrame(all_recipes)
display(df)